In [ ]:
import os
os.chdir('..')

import pandas as pd
import numpy as np
import torch

from prettytable import PrettyTable

from recpack.preprocessing.preprocessors import DataFramePreprocessor
from recpack.preprocessing.filters import Deduplicate, MinRating, MinItemsPerUser
from recpack.scenarios import StrongGeneralization

from hyperopt import fmin, tpe, hp

# helpers & metrics
from src.helper_functions.data_formatting import *
from src.helper_functions.metrics_accuracy import *
from src.helper_functions.metrics_coverage import *
from src.helper_functions.metrics_exposure import *

# models
from src.recommenders.ease import myEASE
from src.recommenders.slim_bn import BNSLIM
from src.recommenders.fslr import FSLR
from src.recommenders.slim_bn_admm import BNSLIM_ADMM
from src.recommenders.mf_fair import FairMF

import os, json
import time
# import pickle

In [ ]:
ratings = pd.read_csv("coco/COCO.csv")

# Selecting the required columns and rows with a single country
ratings = ratings[ratings['country'].str.count('\|') == 0][['user', 'item', 'rating', 'country']]

# https://www.gov.uk/government/publications/countries-defined-as-developing-by-the-oecd/countries-defined-as-developing-by-the-oecd

developing_countries = [
    "Algeria", "Argentina", "Armenia", "Bangladesh", "Belize", "Bolivia",
    "Brazil", "China", "Côte D’Ivoire", "Egypt", "Ethiopia", "Fiji",
    "Guatemala", "India", "Indonesia", "Iran", "Jordan", "Lebanon",
    "Malawi", "Malaysia", "Mexico", "Morocco", "Mozambique", "Nepal",
    "Pakistan", "Papua New Guinea", "Philippines", "Serbia", "South Africa",
    "Sri Lanka", "Surinam", "Syria", "Thailand", "Timor-Leste", "Tonga",
    "Turkey", "Ukraine", "Uzbekistan", "Venezuela", "Zimbabwe"
]

ratings['developing'] = ratings['country'].apply(lambda x: 1 if x in developing_countries else 0)

ratings.head()

In [ ]:
ratings_pp = DataFramePreprocessor("item", "user")

# define filters
deduplicate = Deduplicate("item", "user")
min_rating_filter = MinRating(5, "rating")
min_items_per_user_filter = MinItemsPerUser(10, "item", "user")

# add filters to pre-processor
ratings_pp.add_filter(deduplicate)
ratings_pp.add_filter(min_rating_filter)
ratings_pp.add_filter(min_items_per_user_filter)

# create interaction matrix object
im = ratings_pp.process(ratings)

# apply filters to ratings frame directly
ratings = min_items_per_user_filter.apply(min_rating_filter.apply(deduplicate.apply(ratings)))

In [ ]:
ratings.head()

In [ ]:
# compute sparsity after filtering
sparsity = 1 - im.density

# calculate user interaction and item popularity ranges
user_interactions = im.binary_values.sum(axis=1)
item_popularities = im.binary_values.sum(axis=0)
print(f"User interaction ranges from {user_interactions.min()} to {user_interactions.max()}. Item popularity ranges from {item_popularities.min()} to {item_popularities.max()}.")

# get the raw ids of all items involved
raw_iids = get_raw_item_ids(ratings_pp, im.active_items)

# create iid - gender mapping df
gender_papping_df = ratings[ratings["item"].isin(raw_iids)][["item", "developing"]].drop_duplicates()

# get the raw/inner ids of all females involved
raw_iids_np = gender_papping_df.loc[gender_papping_df["developing"] == 0, "item"].to_numpy()
inner_iids_np = get_inner_item_ids(ratings_pp, raw_iids_np)

# get the raw/inner ids of all males involved
raw_iids_p = gender_papping_df.loc[gender_papping_df["developing"] == 1, "item"].to_numpy()
inner_iids_p = get_inner_item_ids(ratings_pp, raw_iids_p)

items_dict = {
    "protected": inner_iids_p,
    "non-protected": inner_iids_np
}

num_interactions_np, num_interactions_p = im.binary_values[:, inner_iids_np].sum(), im.binary_values[:, inner_iids_p].sum()

# table stats
statTable1 = PrettyTable(["data set","|U|","|I|","int(I)","sparsity"])
statTable1.add_row(["COCO", str(im.num_active_users), str(im.num_active_items), str(im.num_interactions), str(round(sparsity*100,2))])
print(statTable1)

statTable2 = PrettyTable(["data set","attribute","|developing|","int(developing)","|developed|","int(developed)"])
statTable2.add_row(["COCO", "country", str(len(raw_iids_p)), str(num_interactions_p), str(len(raw_iids_np)), str(num_interactions_np)])
print(statTable2)

In [ ]:
# define K for Top-K
K = 10

# Define alpha, the parameter that balances the importance of NDCG and BDV in the objective function.
# Setting alpha = 0.5 gives equal weight to both metrics, aiming to balance relevance (NDCG) and fairness (BDV).
# Adjusting alpha allows for prioritizing one metric over the other.
# For instance, setting alpha closer to 1.0 would prioritize NDCG (accuracy), while setting it closer to 0.0 would prioritize BDV (fairness).
alpha = 0.2

# define seed; seeds tested (1452, 1994, 42, 7, 13800)
SEED = 7

# define scenario
scenario = StrongGeneralization(validation=True, seed=SEED)
scenario.split(im)

# define time threshold
SECONDS = 24*3600

# define number of evaluations
EVALUATIONS = 50

In [ ]:
def accuracy_objective(model, fit_args={}):
    model.fit(scenario.validation_training_data.binary_values, **fit_args)

    valid_rows = scenario.validation_data_in.binary_values.sum(axis=1).A1 != 0
    filtered_validation_data_in = scenario.validation_data_in.binary_values[valid_rows]
    filtered_validation_data_out = scenario.validation_data_out.binary_values[valid_rows]

    # generate predictions and mask training interactions
    predictions = model.predict(filtered_validation_data_in).toarray()
    predictions[filtered_validation_data_in.nonzero()] = -np.inf

    ndcg, _ = tndcg_at_n(predictions, filtered_validation_data_out, K)

    return 1-ndcg

def combined_objective(model, fit_args={}):
    model.fit(scenario.validation_training_data.binary_values, **fit_args)

    valid_rows = scenario.validation_data_in.binary_values.sum(axis=1).A1 != 0
    filtered_validation_data_in = scenario.validation_data_in.binary_values[valid_rows]
    filtered_validation_data_out = scenario.validation_data_out.binary_values[valid_rows]

    predictions = model.predict(filtered_validation_data_in).toarray()
    predictions[filtered_validation_data_in.nonzero()] = -np.inf

    ndcg, _ = tndcg_at_n(predictions, filtered_validation_data_out, K)
    bdv = bdv_at_n(predictions, items_dict, K)

    return alpha * (1-ndcg) + (1 - alpha) * bdv

In [ ]:
# for FairMF
sst_field = torch.zeros((im.num_active_users, im.num_active_items), dtype=torch.bool)
sst_field[:, inner_iids_p] = True

In [ ]:
# optimize ease
optimisation_results_ease = fmin(
    fn=lambda param: accuracy_objective(myEASE(l2=param["l2"])),
    space={"l2": hp.loguniform("l2", np.log(1e0), np.log(1e4))},
    algo=tpe.suggest,
    timeout = SECONDS,
    max_evals = EVALUATIONS,
)

# optimize fslr
optimisation_results_fslr = fmin(
    fn=lambda param: combined_objective(FSLR(l1=param["l1"], l2=param["l2"]), {"inner_ids_pr": inner_iids_p, "inner_ids_npr": inner_iids_np}),
    space={"l1": hp.loguniform("l1", np.log(1e-3), np.log(1e1)),
           "l2": hp.loguniform("l2", np.log(1e0), np.log(1e4))},
    algo=tpe.suggest,
    timeout=SECONDS,
    max_evals=EVALUATIONS
)

# optimize bnslim
optimisation_results_bnslim = fmin(
    fn=lambda param: combined_objective(BNSLIM(knn=100, l1=param["l1"], l2=param["l2"], l3=param["l3"], seed=SEED), {"inner_ids_npr": inner_iids_np}),
    space={"l1": hp.loguniform("l1", np.log(1e-3), np.log(7)),
           "l2": hp.loguniform("l2", np.log(1e-3), np.log(7)),
           "l3": hp.loguniform("l3", np.log(1e1), np.log(1e4))
           }, 
    algo=tpe.suggest,
    timeout=SECONDS,
    max_evals=EVALUATIONS
)

# optimize bnslim admm
optimisation_results_bnslim_admm = fmin(
    fn=lambda param: combined_objective(BNSLIM_ADMM(l1=param["l1"], l2=param["l2"], l3=param["l3"]), {"inner_ids_npr": inner_iids_np}),
    space={"l1": hp.loguniform("l1", np.log(1e-3), np.log(50)),
           "l2": hp.loguniform("l2", np.log(1e0), np.log(1e4)),
           "l3": hp.loguniform("l3", np.log(1e-3), np.log(1e3))},
    algo=tpe.suggest,
    timeout = SECONDS,
    max_evals = EVALUATIONS,
)

# optimize FairMF
factor_choices = [32, 64, 128]
optimisation_results_fairmf = fmin(
    fn=lambda param: combined_objective(FairMF(batch_size=im.num_active_users, learning_rate=param["learning_rate"], l2=param["l2"], num_factors=param["num_factors"], seed=SEED), {"sst_field": sst_field}),
    space={"learning_rate": hp.loguniform("learning_rate", np.log(1e-6), np.log(1e0)),
           "l2": hp.loguniform("l2", np.log(1e-6), np.log(1e-1)),
           "num_factors": hp.choice("num_factors", factor_choices)
           },
    algo=tpe.suggest,
    timeout=SECONDS,
    max_evals=EVALUATIONS
)

optimisation_results_fairmf["num_factors"] = factor_choices[optimisation_results_fairmf["num_factors"]]

opt_params = {}
opt_params.update({
    "ease": optimisation_results_ease,
    "fslr": optimisation_results_fslr,
    "bnslim": optimisation_results_bnslim,
    "bnslim_admm": optimisation_results_bnslim_admm,
    "fairmf": optimisation_results_fairmf
})

folder = f"coco/all/{SEED}"; os.makedirs(folder, exist_ok=True)
with open(f"{folder}/opt_params.json", "w") as f: json.dump(opt_params, f, indent=4)

In [ ]:
with open(f"coco/all/{SEED}/opt_params.json", "r") as f: opt_params = json.load(f)

def initialize_models(opt_params):
    return {
        "ease": myEASE(l2=opt_params["ease"]["l2"]),
        "bnslim": BNSLIM(knn=100, l1=opt_params["bnslim"]["l1"], l2=opt_params["bnslim"]["l2"], l3=opt_params["bnslim"]["l3"], maxIter=50, seed=SEED),
        "fslr": FSLR(l1=opt_params["fslr"]["l1"], l2=opt_params["fslr"]["l2"]),
        "bnslim_admm": BNSLIM_ADMM(l1=opt_params["bnslim_admm"]["l1"], l2=opt_params["bnslim_admm"]["l2"], l3=opt_params["bnslim_admm"]["l3"]),
        "fairmf": FairMF(batch_size=im.num_active_users, l2=opt_params["fairmf"]["l2"], learning_rate=opt_params["fairmf"]["learning_rate"], num_factors=opt_params["fairmf"]["num_factors"], seed=SEED),
    }

# initialize models
models = initialize_models(opt_params)

# define list sizes and metrics
list_sizes = [10, 20, 50, 100]
metrics = ["ndcg", "recall", "bdv", "apcr"]

# initialize results dictionary
results = {
    "iters_num": {model: 0 for model in ["bnslim", "fslr", "bnslim_admm", "fairmf"]},
    "fit_time": {model: 0 for model in models.keys()},
    **{metric: {model: {size: {"mean": 0, "std": 0} if metric in ["ndcg", "recall"] else 0 for size in list_sizes} for model in models.keys()} for metric in metrics}
}

In [ ]:
for model_name, model in models.items():
    
    print(f"Training model {model_name}...")

    params = {}
    if model_name == "fslr":
        params = {"inner_ids_pr": inner_iids_p, "inner_ids_npr": inner_iids_np}
    elif model_name in ["bnslim", "bnslim_admm"]:
        params = {"inner_ids_npr": inner_iids_np}
    elif model_name == "fairmf":
        params = {"sst_field": sst_field}
    
    start_time = time.time()
    model.fit(scenario.full_training_data.binary_values, **params)
    results["fit_time"][model_name] = time.time() - start_time

    if model_name in results["iters_num"]:
        if model_name == "fairmf":
            results["iters_num"][model_name] = model.epochs
        else:
            results["iters_num"][model_name] = model.iters

    valid_rows = scenario.test_data_in.binary_values.sum(axis=1).A1 != 0 # delete zero rows (strong gen)
    filtered_test_data_in = scenario.test_data_in.binary_values[valid_rows]
    filtered_test_data_out = scenario.test_data_out.binary_values[valid_rows]

    # generate predictions and mask training interactions
    predictions = model.predict(filtered_test_data_in).toarray()
    predictions[filtered_test_data_in.nonzero()] = -np.inf

    # compute evaluation metrics for different values of K
    for K in list_sizes:
        # accuracy metrics
        results["ndcg"][model_name][K]["mean"], results["ndcg"][model_name][K]["std"] = tndcg_at_n(predictions, filtered_test_data_out, K)
        results["recall"][model_name][K]["mean"], results["recall"][model_name][K]["std"] = recall_at_n(predictions, filtered_test_data_out, K)

        # fairness metrics
        results["bdv"][model_name][K] = bdv_at_n(predictions, items_dict, K)
        results["apcr"][model_name][K] = apcr_at_n(predictions, items_dict, K)

    # # save model
    # pickle.dump(model, open(f"coco/all/{SEED}/{model_name}.pkl", "wb"))

# save results
with open(f"coco/all/{SEED}/results.json", "w") as f: json.dump(results, f, indent=4)